# SciKera Classification

`pip install scikeras[tensorflow]`


In [1]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from scikeras.wrappers import KerasClassifier

In [2]:
# Read data
iris = datasets.load_iris()

# Create a DataFrame
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df["target"] = iris.target

# Shuffle the data
df = df.sample(frac=1, random_state=0)

# Extract X and y
X = df.iloc[:, 0:4].values
y = df["target"].values

# Split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0, stratify=y
)
# Standardization
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [3]:
def get_clf_model(hidden_layer_sizes, meta, compile_kwargs):

    model = tf.keras.Sequential()
    input_layer = tf.keras.layers.Input(shape=(meta["n_features_in_"]))
    model.add(input_layer)

    for i, hidden_layer_size in enumerate(hidden_layer_sizes):
        hidden_layer = tf.keras.layers.Dense(
            units=hidden_layer_size, activation="relu", name=f"hidden{i+1}"
        )
        model.add(hidden_layer)

    if meta["target_type_"] == "binary":
        n_output_units = 1
        output_activation = "sigmoid"
        loss = "binary_crossentropy"
    elif meta["target_type_"] == "multiclass":
        n_output_units = meta["n_classes_"]
        output_activation = "softmax"
        loss = "sparse_categorical_crossentropy"
    else:
        raise NotImplementedError(f"Unsupported task type: {meta['target_type_']}")

    output_layer = tf.keras.layers.Dense(n_output_units, activation=output_activation)
    model.add(output_layer)

    model.compile(
        optimizer=compile_kwargs["optimizer"],
        loss=loss,
        metrics=["accuracy"],
    )

    return model

In [4]:
clf = KerasClassifier(
    model=get_clf_model,
    hidden_layer_sizes=(16, 3),
    optimizer="adam",
    optimizer__learning_rate=0.001,
    batch_size=16,
    epochs=200,
    validation_split=0.2,
    verbose=1,
)

In [5]:
clf.fit(X_train_std, y_train)

Epoch 1/200


2024-03-01 15:18:09.420756: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-03-01 15:18:09.420779: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-03-01 15:18:09.420784: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-03-01 15:18:09.420975: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-01 15:18:09.420992: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-03-01 15:18:09.717582: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - ETA: 0s - loss: 1.4456 - accuracy: 0.0714

2024-03-01 15:18:11.506978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 2s 129ms/step - loss: 1.4456 - accuracy: 0.0714 - val_loss: 1.3761 - val_accuracy: 0.0476
Epoch 2/200
6/6 [==============================] - 0s 9ms/step - loss: 1.3857 - accuracy: 0.0833 - val_loss: 1.3151 - val_accuracy: 0.0476
Epoch 3/200
6/6 [==============================] - 0s 9ms/step - loss: 1.3281 - accuracy: 0.0833 - val_loss: 1.2573 - val_accuracy: 0.1905
Epoch 4/200
6/6 [==============================] - 0s 9ms/step - loss: 1.2755 - accuracy: 0.0952 - val_loss: 1.2014 - val_accuracy: 0.1905
Epoch 5/200
6/6 [==============================] - 0s 9ms/step - loss: 1.2247 - accuracy: 0.1310 - val_loss: 1.1510 - val_accuracy: 0.2381
Epoch 6/200
6/6 [==============================] - 0s 9ms/step - loss: 1.1817 - accuracy: 0.1905 - val_loss: 1.1057 - val_accuracy: 0.2857
Epoch 7/200
6/6 [==============================] - 0s 10ms/step - loss: 1.1388 - accuracy: 0.2619 - val_loss: 1.0636 - val_accuracy: 0.2857
Epoch 8/200
6/6 [===================

KerasClassifier(
	model=<function get_clf_model at 0x29458f920>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=adam
	loss=None
	metrics=None
	batch_size=16
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.2
	shuffle=True
	run_eagerly=False
	epochs=200
	hidden_layer_sizes=(16, 3)
	optimizer__learning_rate=0.001
	class_weight=None
)

In [6]:
clf.score(X_test_std, y_test)

1/3 [=========>....................] - ETA: 0s

2024-03-01 15:18:22.097102: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 0s 96ms/step


0.9333333333333333